In [1]:
import telebot
from time import sleep
import sqlite3
import random
import csv
import re

In [2]:
conn = sqlite3.connect('Swadesh.db', check_same_thread=False)
cur = conn.cursor()

In [3]:
# Функция для записи позиции списка во временный csv файл
def to_csv(file_name, line):
    lst = []
    for n, el in enumerate(line):
        for nn, w in enumerate(el.split('  ')):
            if w:
                lst.append([n, nn, ' '.join(w.split()[:-1]), w.split()[-1]])
            else:
                lst.append([n, nn])

    with open(file_name, 'w', encoding='UTF-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(lst)

In [4]:
# Функция для записи изменений, вносимых пользователем во временный csv файл
def changes_to_csv(file_name, lst):
    with open(file_name, 'r', encoding='UTF-8', newline='') as f:
        r = csv.reader(f, delimiter=',')
        old_words = [row for row in r]
        words = [[row[2], row[3]] for row in old_words if len(row) > 2]
        nules = len([w for w in words if not int(w[1])])
        cognation = max([int(el[1]) for el in words]) + 1
        words = [el[0] for el in words if not int(el[1])]
        
        if lst != [0]:
            words_to_change = [words[n-1] for n in lst]
            
            for w in old_words:
                if len(w) > 2:
                    if w[2] in words_to_change:
                        w[3] = cognation
    
            if nules - len(lst) == 1:
                for w in old_words:
                    if len(w) > 2:
                        if int(w[3]) == 0:
                            w[3] = cognation + 1
        
        else:
            i = 1
            for w in old_words:
                if len(w) > 2:
                    if w[2] in words:
                        w[3] = cognation + i
                        i += 1
            
    with open(file_name, 'w', encoding='UTF-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(old_words)

    if lst != [0]:
        if nules - len(lst) == 1:
            return 0
        else:
            with open(file_name, 'r', encoding='UTF-8', newline='') as f:
                r = csv.reader(f, delimiter=',')
                res = [w[2] for w in r if len(w) > 2 and not int(w[3])]
                return res
    
    else:
        return 0

In [5]:
# Функция для сбора слов в список
def to_list(string):
    tmp = []
    for el in str(string).split():
        if '-' not in el:
            tmp.append(int(el))
        else:
            elel = list(map(int, el.split('-')))
            for n in range(min(elel), max(elel)+1):
                tmp.append(n)
    return list(set(tmp))

In [6]:
# Функция для вывода содержимого временного csv файла в список
def csv_to_list(filename):
    with open(filename, 'r', encoding='UTF-8', newline='') as f:
        r = csv.reader(f, delimiter=',')
        ll = []
        l = []
        s = 0
        for row in r:
            if int(row[0]) == s:
                if len(row) > 2:
                    l.append(f'{row[2]} {row[3]}')
                else:
                    l.append('')
            else:
                ll.append('  '.join(l))
                l = []
                if len(row) > 2:
                    l.append(f'{row[2]} {row[3]}')
                else:
                    l.append('')
                s += 1
        ll.append('  '.join(l))
        return ll

Получим список имеющихся списков

In [7]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
lists = [list[0] for list in cur.fetchall() if list[0][0].isupper()]
lists_commands = ''.join([f"/{l}\n" for l in lists])

In [9]:
bot = telebot.TeleBot(TOKEN)

In [10]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.send_message(message.chat.id, """
    Здравствуйте! Я бот, в котором Вы можете поразмечать списки Сводеша. 
Если хотите приступить к разметке, воспользуйтесь командой /lists
    """)
    
    #     занесём участника в таблицу editors
    try:
            cur.execute(f"""
                INSERT INTO editors
                (username, telegram_id)
                VALUES
                ('{message.from_user.username}', '{message.from_user.id}');
                """)
            conn.commit()
    except:
        pass

In [11]:
@bot.message_handler(commands=['lists'])
def send_lists(message):
    bot.send_message(message.chat.id, f"""
    Выберите языковую группу, позицию из списка языков которой Вы бы хотели разметить:

{lists_commands}
Или воспользуйтесь командой /help, чтобы прочитать инструкцию к разметке
""")

In [12]:
@bot.message_handler(commands=['help'])
def send_help(message):
    bot.send_message(message.chat.id, """
    Разметка происходит следующим образом:
    
Бот присылает Вам позицию из выбранного Вами списка, представленную несколькими словами с одинаковым значением, например:
    1) hrudź
    2) prsi
    3) ňadro
    4) грудь
    5) гру́дзі
    6) гърда
    7) pierś
    8) bok

Вы отправляете боту номера тех слов, которые являются исторически родственными, в нашем случае это, например:
    1 4-6
Номера следует указывать через пробел, если они не соседствуют, промежутки оформляются знаком "-"

Далее бот отправит Вам новый список, в котором будут отсутствовать выбранные Вами на предыдущем шаге элементы:
    1) prsi
    2) ňadro
    3) pierś
    4) bok

Если в списке остались ещё группы родственных слов, отправьте боту новую группу чисел, в нашем случае это будет
    1 3

После чего вы получите новый список:
    1) ňadro
    2) bok
В котором все элементы уникальны. В таком случае просто напишите боту '0'
Если после последнего Вашего набора номеров должен будет оставаться список из одного элемента, бот не будет выводить его Вам на экран, а разберётся с ним сам.

В конце бот непременно Вас поблагодарит и предложит разметить ещё одну строку из списка на выбор.
Если в процессе разметки Вы решите, что по какой-то причине не готовы её заканчивать, воспользуйтесь командой /cancel

Чтобы приступить к разметке, воспользуйтесь командой /lists
""")

In [13]:
@bot.message_handler(commands=lists)
def send_line(message, group_list=None, line_id = None):
    if not group_list:
        group_list = message.text[1:]

#     Получаем id пользователя
    cur.execute(f"SELECT id FROM editors WHERE telegram_id={message.from_user.id};")
    user_id = cur.fetchone()
    if not user_id:
        bot.send_message(message.chat.id, f"""Для начала работы я настоятельно рекомендую Вам воспользоваться командой /start""")
    else:
        user_id = user_id[0]
        
#         Получаем статус последней правки пользователя, если таковая имеется
        cur.execute(f"SELECT state FROM edits WHERE user_id={user_id};")
        state = cur.fetchall()

#         Проверяем, можно ли выдать строку
        if state and not state[-1][0]:
            bot.send_message(message.chat.id, f"""
К сожалению, Вы не закончили работу с предыдущей выпавшей Вам строкой.
Пожалуйста, прежде, чем брать новую строку, закончите разметку имееющейся, или откажитесь, использовав команду /cancel
            """)
        else:
            try:
#                 Получим строку, если это не переразметка командой /cancel
                if not line_id:
#                     Обращаемся к таблице, которую выбрал пользователь и достаём из неё случайную неразмеченную строку 
                    cur.execute(f"SELECT id FROM {group_list} WHERE state=-1;")

#                     Выбираем номер случайной неразмеченной строки
                    line_id = random.choice([l[0] for l in cur.fetchall()])

#                 Записываем данные о правке в соответствующую таблицу
                cur.execute(f"""
INSERT INTO edits
(user_id, state, language_group, line)
VALUES
('{user_id}', '0', '{group_list}', '{line_id}');
                """)
                conn.commit()

#                 Изменим статус строки, чтобы она не досталась ещё одному пользователю
                cur.execute(f""" UPDATE {group_list} SET state = 0 WHERE id = {line_id};""")
                conn.commit()

#                 Выведем эту строку на экран
                cur.execute(f"SELECT * FROM {group_list} WHERE id={line_id};")
                words = []
                els = cur.fetchone()[2:]
                for el in els:
                    if el:
                        for word in el.split('  '):
                            words.append(word[:-2])

                words_list = ''.join([f'{n}) {word}\n' for n, word in enumerate(words, 1)])
                bot.send_message(message.chat.id, f"""
Вот строка, выпавшая Вам для разметки:
{words_list}
Чтобы отменить разметку данной строки, воспользуйтесь командой /cancel
                """)

#                 Запишем данные в файл
                to_csv(f'{message.from_user.id}.csv', els)

            except IndexError:
                bot.send_message(message.chat.id, f"""
К сожалению, в списке {group_list} не осталось неразмеченных строк. 
Выберите другой список, воспользовавшись командой /lists
                """)

In [14]:
@bot.message_handler(regexp='([0-9]{1,2}[ |-]?)+')
def get_answer(message):
#     Получаем id пользователя
    cur.execute(f"SELECT id FROM editors WHERE telegram_id={message.from_user.id};")
    user_id = cur.fetchone()
    if not user_id:
        bot.send_message(message.chat.id, f"""Для начала работы я настоятельно рекомендую Вам воспользоваться командой /start""")
    else:
        user_id = user_id[0]

#         Получаем информацию о последней, выпавшей ему для разметки строке
        cur.execute(f"SELECT * FROM edits WHERE user_id={user_id};")
        info = cur.fetchall()
    
        if not info:
            bot.send_message(message.chat.id, f"""
Кажется, Вы ещё не получили строку, но уже торопитесь её разметить. 
Достойное похвалы рвение, но я работаю в другом порядке.
Чтобы получить строку, воспользуйтесь командой /lists
                """)
        else:
            state, group_list, line_id = info[-1][2:]
            if state == 1:
                bot.send_message(message.chat.id, f"""
Кажется, Вы уже разметили строку и отправили её в базу данных.
Чтобы получить новую строку, воспользуйтесь командой /lists
                """)

            elif state == -1:
                bot.send_message(message.chat.id, f"""
Кажется, Вы отменили разметку выпавшей Вам строки.
Чтобы получить новую строку, воспользуйтесь командой /lists
            """)
#             Обработаем ответ пользователя
            else:
                changes = changes_to_csv(f'{message.from_user.id}.csv', to_list(message.text))
                if not changes:
                    
#                     Записываем данные об успешной правке
                    cur.execute(f"""
INSERT INTO edits
(user_id, state, language_group, line)
VALUES
('{user_id}', '1', '{group_list}', '{line_id}');
                    """)
                    conn.commit()
        
#                     Изменим статус строки, чтобы она больше не была доступна другим пользователям для разметки
                    cur.execute(f""" UPDATE {group_list} SET state = 1 WHERE id = {line_id};""")
                    conn.commit()
        
#                     Запишем непосредственно саму правку
                    cur.execute(f"PRAGMA table_info({group_list})")
                    columns = [name[1] for name in cur.fetchall() if name[2] == 'TEXT']
                    
                    final_list = csv_to_list(f'{message.from_user.id}.csv')
                    
                    for cnt, lang in enumerate(columns):
                        cur.execute(f""" UPDATE {group_list} SET '{lang}' = '{final_list[cnt]}' WHERE id = {line_id};""")
                        conn.commit()
                    
                    bot.send_message(message.chat.id, f"""
Поздравляю, Вы завершили разметку выпавшей Вам строки.
Чтобы получить новую строку, воспользуйтесь командой /lists
                    """)
                    
                else:
                    words_list = ''.join([f'{n}) {word}\n' for n, word in enumerate(changes, 1)])
                    bot.send_message(message.chat.id, f"""
Я только что вычеркнул из строки слова, которые Вы определили, как родственные. Вот, что у меня вышло:

{words_list}
Ожидаю от вас продолжения разметки!
Чтобы отменить разметку данной строки, воспользуйтесь командой /cancel
                    """)

In [15]:
@bot.message_handler(commands=['cancel'])
def send_cancel(message):
#     Получаем id пользователя
    cur.execute(f"SELECT id FROM editors WHERE telegram_id={message.from_user.id};")
    user_id = cur.fetchone()
    if not user_id:
        bot.send_message(message.chat.id, f"""Для начала работы я настоятельно рекомендую Вам воспользоваться командой /start""")
    else:
        user_id = user_id[0]

#         Получаем информацию о последней, выпавшей ему для разметки строке
        cur.execute(f"SELECT * FROM edits WHERE user_id={user_id};")
        info = cur.fetchall()

        if not info:
            bot.send_message(message.chat.id, f"""
Кажется, Вам нечего отменять. 
Хороший повод разметить пару строк.
Для этого воспользуйтесь командой /lists
                """)
        else:
            state, group_list, line_id = info[-1][2:]
#             Добавим возможность переразметки строки
            if state == 1:
                bot.send_message(message.chat.id, f"""
Вы решили переразметить последнюю отправленую Вами строку. 
К сожалению, я пока так не умею. Вы можете разметить другую строку.
Для этого воспользуйтесь командой /lists
        """)
#                 bot.send_message(message.chat.id, f"""
# Вы решили переразметить последнюю отправленую Вами строку. 
# В следующем сообщении я отправлю вам её снова
#         """)
#                 send_line(message, group_list=group_list, line_id = line_id)
            elif state == -1:
                bot.send_message(message.chat.id, f"""
Кажется, Вы только что пытались отменить отмену разметки строки. 
Боюсь, я не в силах отменить Вашу отмену, но Вы можете разметить какую-нибудь другую строку.
Для этого воспользуйтесь командой /lists
                """)
            else:
#                 Записываем данные об отмене правки
                cur.execute(f"""
INSERT INTO edits
(user_id, state, language_group, line)
VALUES
('{user_id}', '-1', '{group_list}', '{line_id}');
        """)
                conn.commit()

#                 Изменим статус строки, чтобы она снова была доступна другим пользователям для разметки
                cur.execute(f""" UPDATE {group_list} SET state = -1 WHERE id = {line_id};""")
                conn.commit()

                bot.send_message(message.chat.id, f"""
Вы решили не размечать данную строку. 
Возможно, вы хотите разметить строку из другого списка. Для этого воспользуйтесь командой /lists
                """)

In [16]:
@bot.message_handler(func=lambda m: True)
def send_warning(message):
    bot.send_message(message.chat.id, f"""
Увы, я не понял, что Вы написали.
Чтобы получить строку для разметки, воспользуйтесь командой /lists
Чтобы узнать, как меня использовать, прочитайте /help
                    """)

In [ ]:
while True:
    try:
        bot.polling(none_stop=True)
    except Exception as e:
        print(e)
        sleep(2)

invalid literal for int() with base 10: '1+3'
near "1": syntax error
near "1": syntax error
near "1": syntax error
